# 전처리 1
- 아파트 실거래가 관련 변수 추가
## 1. 경제 관련 데이터
- 기준금리
- GDP 성장률

## 2. 거래 데이터
- 소비심리지수
- 아파트 매매 실거래지수
- 최근 실거래 데이터
- 최근 전세 데이터
- 최근 3달동안의 거래량

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import warnings
from tqdm import tqdm
from datetime import datetime

plt.rcParams['font.family'] = 'Malgun Gothic'
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
warnings.filterwarnings(action='ignore')

# 데이터 불러오기

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

train['train_test'] = 'train'
test['train_test'] = 'test'

data = pd.concat([train, test], axis = 0)

In [3]:
data.to_csv('../data/raw_data.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6183 entries, 0 to 195
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      6183 non-null   object 
 1   sigungu                 6183 non-null   object 
 2   jibun                   6183 non-null   object 
 3   apt_name                6183 non-null   object 
 4   exclusive_use_area      6183 non-null   float64
 5   transaction_year_month  6183 non-null   int64  
 6   transaction_day         6183 non-null   int64  
 7   transaction_real_price  5987 non-null   float64
 8   floor                   6183 non-null   int64  
 9   year_of_completion      6183 non-null   int64  
 10  train_test              6183 non-null   object 
dtypes: float64(2), int64(4), object(5)
memory usage: 579.7+ KB


In [5]:
### 날짜 데이터 전처리

def preprocess_tran_date(x):
    if type(x) == int:
        if x < 10:
            return '0'+str(x)
        else:
            return str(x)
    else:
        return x
    
# 데이터 병합을 위해 transaction_date 컬럼 생성
data['transaction_day'] = data['transaction_day'].apply(preprocess_tran_date)
data['transaction_date'] = data['transaction_year_month'].astype(int).astype(str) + data['transaction_day'].astype(str)
data['transaction_date'] = pd.to_datetime(data['transaction_date'])

In [6]:
data['transaction_year_month'] = data['transaction_year_month'].astype('str')

In [7]:
data.dtypes

id                                object
sigungu                           object
jibun                             object
apt_name                          object
exclusive_use_area               float64
transaction_year_month            object
transaction_day                   object
transaction_real_price           float64
floor                              int64
year_of_completion                 int64
train_test                        object
transaction_date          datetime64[ns]
dtype: object

## 데이터 병합하기
### 소비심리지수
- 데이터가 한 달 뒤에 생성되는 점을 고려하여, 이전 달 데이터를 사용해 학습 진행

- 기간 열을 기준으로 데이터를 병합함

In [8]:
consume = pd.read_csv('../data/부동산시장_소비심리지수.csv', encoding = 'cp949')
df_consume = consume.T.reset_index().iloc[2:]
df_consume.columns = ['기간','소비심리지수']

In [9]:
df_consume.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 2 to 116
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   기간      115 non-null    object
 1   소비심리지수  115 non-null    object
dtypes: object(2)
memory usage: 1.9+ KB


In [10]:
df_consume.head()

,기간,소비심리지수
2,2013.12,112.6
3,2014.01,128.8
4,2014.02,132.6
5,2014.03,117.1
6,2014.04,116.2


In [11]:
data['transaction_year_month']

0      201401
1      201401
2      201401
3      201402
4      201402
        ...  
191    202306
192    202306
193    202306
194    202306
195    202303
Name: transaction_year_month, Length: 6183, dtype: object

In [12]:
# '기간' 열을 datetime으로 변환
df_consume['기간'] = pd.to_datetime(df_consume['기간'])

# 데이터 병합을 위해 data 기간과 맞춤
df_consume['기간'] = df_consume['기간'] + pd.DateOffset(months=1)

# 연도와 월만 출력하기 위해 문자열 형태로 변환
df_consume['기간'] = df_consume['기간'].dt.strftime('%Y%m')

In [13]:
df_consume.dtypes

기간        object
소비심리지수    object
dtype: object

In [14]:
data['transaction_year_month'].iloc[:5]

0    201401
1    201401
2    201401
3    201402
4    201402
Name: transaction_year_month, dtype: object

In [15]:
data = data.merge(df_consume,  left_on='transaction_year_month', right_on='기간')

In [16]:
data.drop(['기간'], axis = 1, inplace = True)

In [17]:
data.columns

Index(['id', 'sigungu', 'jibun', 'apt_name', 'exclusive_use_area',
       'transaction_year_month', 'transaction_day', 'transaction_real_price',
       'floor', 'year_of_completion', 'train_test', 'transaction_date',
       '소비심리지수'],
      dtype='object')

### 아파트 매매 실거래지수
- 데이터가 한 달 뒤에 생성되는 점을 고려하여, 이전 달 데이터를 사용해 학습을 진행한다.

- 기간 열을 기준으로 데이터를 병합함

In [18]:
trading_index = pd.read_excel('../data/유형별_매매가격지수.xlsx')

df_trading_index = trading_index.iloc[4:]
df_trading_index.columns = ['기간','아파트 매매 실거래 가격 지수']

In [19]:
df_trading_index.dtypes

기간                  object
아파트 매매 실거래 가격 지수    object
dtype: object

In [20]:
# '기간' 열을 datetime으로 변환
df_trading_index['기간'] = pd.to_datetime(df_trading_index['기간'])

# 데이터 병합을 위해 data 기간과 맞춤
df_trading_index['기간'] = df_trading_index['기간'] + pd.DateOffset(months=1)

# 연도와 월만 출력하기 위해 문자열 형태로 변환합니다.
df_trading_index['기간'] = df_trading_index['기간'].dt.strftime('%Y%m')

In [21]:
df_trading_index[:6]

,기간,아파트 매매 실거래 가격 지수
4,201401,66.6
5,201402,66.8
6,201403,67.3
7,201404,67.9
8,201405,67.8
9,201406,67.8


In [22]:
data = data.merge(df_trading_index,  left_on='transaction_year_month', right_on='기간')

In [23]:
data.drop(['기간'], axis = 1, inplace = True)

In [24]:
data.head(1)

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,train_test,transaction_date,소비심리지수,아파트 매매 실거래 가격 지수
0,TRAIN_0000,서울특별시 강남구 대치동,503,개포우성1,158.54,201401,07,174000.0,13,1983,train,2014-01-07,112.6,66.6


### 기준금리
- 변동일자를 기준으로 일자에 맞는 기준금리 데이터 넣기

In [25]:
df_interest = pd.read_excel('../data/한국은행 기준금리 및 여수신금리_변동일자기준.xlsx')
df_interest = df_interest.sort_values(by=['년도'])
df_interest.head()

,년도,변동일자,기준금리
22,2012,10월 11일,2.75
21,2013,05월 09일,2.50
20,2014,08월 14일,2.25
19,2014,10월 15일,2.00
18,2015,03월 12일,1.75


In [26]:
df_interest.dtypes

년도        int64
변동일자     object
기준금리    float64
dtype: object

In [27]:
df_interest['변동일자'] = pd.to_datetime(df_interest['변동일자'], format='%m월 %d일')

In [28]:
# '변동일자' 열의 날짜 형식을 'MM-DD'로 변경
df_interest['변동일자'] = df_interest['변동일자'].dt.strftime('%m-%d')

In [29]:
# '년도'와 '변동일자' 열을 합쳐 '기간' 열 생성
df_interest['기간'] = df_interest['년도'].astype(str) + '-' + df_interest['변동일자']

# '기간' 열의 형식을 datetime으로 변경
df_interest['기간'] = pd.to_datetime(df_interest['기간'])

In [30]:
display(
    df_interest.head(),
    df_interest.dtypes
)

,년도,변동일자,기준금리,기간
22,2012,10-11,2.75,2012-10-11
21,2013,05-09,2.50,2013-05-09
20,2014,08-14,2.25,2014-08-14
19,2014,10-15,2.00,2014-10-15
18,2015,03-12,1.75,2015-03-12


년도               int64
변동일자            object
기준금리           float64
기간      datetime64[ns]
dtype: object

In [31]:
interest_date = df_interest['기간']
interest = df_interest['기준금리']

In [32]:
interest_date

22   2012-10-11
21   2013-05-09
20   2014-08-14
19   2014-10-15
18   2015-03-12
        ...    
4    2022-07-13
3    2022-08-25
2    2022-10-12
1    2022-11-24
0    2023-01-13
Name: 기간, Length: 23, dtype: datetime64[ns]

In [33]:
# date가 정렬되어 있어야 병합이 가능함
data = data.sort_values(by = 'transaction_date')

<strong> <font size=3> merge_asof </strong>
- rf : https://pandas.pydata.org/docs/reference/api/pandas.merge_asof.html
- pandas 라이브러리에서 제공하는 함수로, 두 데이터 프레임을 특정 키 기준으로 가까운 값끼리 병합하는 기능을 제공
- 가까운 키 값 기준 병합: merge_asof는 두 데이터 프레임을 병합할 때, 정확히 일치하는 키 값을 찾는 대신 가장 가까운 키 값을 기준으로 병합함. 시간 데이터와 같이 연속적인 값에서 매우 유용
- left-join과의 유사성: 이 함수는 left-join과 유사하게 작동하지만, 정확한 일치 대신 가장 가까운 키 값을 찾아서 병합한다는 점에서 차이가 있음
- 금리 변동일자에 맞춰서 기준금리 데이터 넣기

In [34]:
merged_df = pd.merge_asof(data,df_interest, left_on= 'transaction_date', right_on='기간')
merged_df.sample(5)

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,train_test,transaction_date,소비심리지수,아파트 매매 실거래 가격 지수,년도,변동일자,기준금리,기간
79,TRAIN_0403,서울특별시 강남구 대치동,1025,롯데캐슬리베,121.0894,201401,22,119000.0,8,2008,train,2014-01-22,112.6,66.6,2013,05-09,2.50,2013-05-09
1975,TRAIN_2219,서울특별시 강남구 대치동,506,선경1차(1동-7동),136.6800,201604,02,174000.0,3,1983,train,2016-04-02,119.2,75.3,2015,06-11,1.50,2015-06-11
1204,TRAIN_1682,서울특별시 강남구 대치동,511,한보미도맨션2,126.3300,201504,15,153500.0,14,1985,train,2015-04-15,138.7,70.6,2015,03-12,1.75,2015-03-12
5620,TRAIN_5816,서울특별시 강남구 대치동,891-6,테헤란로대우아이빌(891-6),59.8300,202103,31,85000.0,14,2004,train,2021-03-31,124.7,96.8,2020,05-28,0.50,2020-05-28
3464,TRAIN_3926,서울특별시 강남구 대치동,992,현대1,84.2700,201707,10,99000.0,9,1990,train,2017-07-10,125.4,82.4,2016,06-09,1.25,2016-06-09


In [35]:
merged_df.drop(['변동일자','년도','기간'], axis = 1, inplace = True)

In [36]:
merged_df[:3]

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,train_test,transaction_date,소비심리지수,아파트 매매 실거래 가격 지수,기준금리
0,TRAIN_0320,서울특별시 강남구 대치동,670,동부센트레빌,145.83,201401,02,180000.0,16,2005,train,2014-01-02,112.6,66.6,2.5
1,TRAIN_0699,서울특별시 강남구 대치동,986-14,하이캐슬,84.06,201401,02,68000.0,5,2008,train,2014-01-02,112.6,66.6,2.5
2,TRAIN_0705,서울특별시 강남구 대치동,511,한보미도맨션1,161.36,201401,04,155000.0,5,1983,train,2014-01-04,112.6,66.6,2.5


### GDP 성장률
- 분기별로 집계되는 데이터임을 고려하여 전분기 데이터 사용

In [37]:
df_gdp = pd.read_excel('../data/GDP 성장률.xlsx')
df_gdp.head()

,기간,GDP 성장률
0,2012-Q4,0.5
1,2013-Q1,0.9
2,2013-Q2,1.2
3,2013-Q3,0.9
4,2013-Q4,0.9


In [38]:
df_gdp.dtypes

기간          object
GDP 성장률    float64
dtype: object

In [39]:
# period 타입으로 변환
df_gdp['기간'] = pd.to_datetime(df_gdp['기간']).dt.to_period('Q')

In [40]:
df_gdp.dtypes

기간         period[Q-DEC]
GDP 성장률          float64
dtype: object

In [41]:
merged_df['transaction_date'].dt.to_period('Q')

0       2014Q1
1       2014Q1
2       2014Q1
3       2014Q1
4       2014Q1
         ...  
6112    2023Q2
6113    2023Q2
6114    2023Q2
6115    2023Q2
6116    2023Q2
Name: transaction_date, Length: 6117, dtype: period[Q-DEC]

In [42]:
pd.offsets.QuarterBegin()

<QuarterBegin: startingMonth=3>

In [43]:
# '기간' 열을 datetime 타입으로 변환
df_gdp['기간'] = df_gdp['기간'].dt.to_timestamp()

# '거래일자' 열의 전분기 마지막일을 구하기
merged_df['전분기 마지막일'] = merged_df['transaction_date'].dt.to_period('Q').dt.start_time - pd.offsets.QuarterEnd()

# '거래일자'가 속한 전분기의 경제 성장률을 찾아 병합
merged_df2 = pd.merge_asof(merged_df, df_gdp, left_on='전분기 마지막일', right_on='기간')


pd.merge_asof 함수를 사용하므로, 각 거래일자가 속한 전 분기와 가장 가까운 '기간'의 데이터를 찾아 병합

In [44]:
df_gdp[:4]

,기간,GDP 성장률
0,2012-10-01,0.5
1,2013-01-01,0.9
2,2013-04-01,1.2
3,2013-07-01,0.9


In [45]:
merged_df2

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,train_test,transaction_date,소비심리지수,아파트 매매 실거래 가격 지수,기준금리,전분기 마지막일,기간,GDP 성장률
0,TRAIN_0320,서울특별시 강남구 대치동,670,동부센트레빌,145.83,201401,02,180000.0,16,2005,train,2014-01-02,112.6,66.6,2.5,2013-12-31,2013-10-01,0.9
1,TRAIN_0699,서울특별시 강남구 대치동,986-14,하이캐슬,84.06,201401,02,68000.0,5,2008,train,2014-01-02,112.6,66.6,2.5,2013-12-31,2013-10-01,0.9
2,TRAIN_0705,서울특별시 강남구 대치동,511,한보미도맨션1,161.36,201401,04,155000.0,5,1983,train,2014-01-04,112.6,66.6,2.5,2013-12-31,2013-10-01,0.9
3,TRAIN_0660,서울특별시 강남구 대치동,891-6,테헤란로대우아이빌(891-6),33.11,201401,04,27000.0,20,2004,train,2014-01-04,112.6,66.6,2.5,2013-12-31,2013-10-01,0.9
4,TRAIN_0102,서울특별시 강남구 대치동,891-28,대치동우정에쉐르1,59.98,201401,04,37400.0,6,2004,train,2014-01-04,112.6,66.6,2.5,2013-12-31,2013-10-01,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6112,TEST_0161,서울특별시 강남구 대치동,902,포스코더샵,139.54,202306,23,NaN,16,2004,test,2023-06-23,103.7,99.1,3.5,2023-03-31,2023-01-01,0.3
6113,TEST_0194,서울특별시 강남구 대치동,511,한보미도맨션2,126.33,202306,23,NaN,12,1985,test,2023-06-23,103.7,99.1,3.5,2023-03-31,2023-01-01,0.3
6114,TEST_0045,서울특별시 강남구 대치동,63,"대치우성아파트1동,2동,3동,5동,6동,7동",84.69,202306,24,NaN,12,1984,test,2023-06-24,103.7,99.1,3.5,2023-03-31,2023-01-01,0.3
6115,TEST_0013,서울특별시 강남구 대치동,500,개포우성2,84.69,202306,24,NaN,1,1984,test,2023-06-24,103.7,99.1,3.5,2023-03-31,2023-01-01,0.3


In [46]:
df_gdp[-5:]

,기간,GDP 성장률
38,2022-04-01,0.8
39,2022-07-01,0.2
40,2022-10-01,-0.3
41,2023-01-01,0.3
42,2023-04-01,0.6


In [47]:
merged_df2.sample(10)

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,train_test,transaction_date,소비심리지수,아파트 매매 실거래 가격 지수,기준금리,전분기 마지막일,기간,GDP 성장률
5027,TRAIN_4877,서울특별시 강남구 대치동,506,선경2차(8동-12동),84.5500,201910,09,255000.0,9,1985,train,2019-10-09,122.5,91,1.50,2019-09-30,2019-07-01,0.4
477,TRAIN_0409,서울특별시 강남구 대치동,1025,롯데캐슬리베,121.0894,201407,22,124000.0,11,2008,train,2014-07-22,107,67.8,2.50,2014-06-30,2014-04-01,0.9
5204,TRAIN_5421,서울특별시 강남구 대치동,890-49,선릉역대우아이빌(890-49),28.3400,202002,20,26000.0,11,2004,train,2020-02-20,131.6,96.3,1.25,2019-12-31,2019-10-01,1.2
4086,TRAIN_4053,서울특별시 강남구 대치동,670,동부센트레빌,121.7400,201803,10,245000.0,12,2005,train,2018-03-10,126.3,90.9,1.50,2017-12-31,2017-10-01,-0.3
1368,TRAIN_1432,서울특별시 강남구 대치동,316,은마,76.7900,201506,08,92500.0,4,1979,train,2015-06-08,137.8,71.9,1.75,2015-03-31,2015-01-01,0.8
4598,TRAIN_4734,서울특별시 강남구 대치동,1027,래미안대치팰리스,84.9700,201905,26,237000.0,7,2015,train,2019-05-26,92.1,90.5,1.75,2019-03-31,2019-01-01,-0.1
2298,TRAIN_1844,서울특별시 강남구 대치동,1014-3,대치삼성,59.8800,201606,16,79000.0,18,2000,train,2016-06-16,126.4,76,1.25,2016-03-31,2016-01-01,0.3
3305,TRAIN_3592,서울특별시 강남구 대치동,316,은마,84.4300,201706,02,129500.0,1,1979,train,2017-06-02,126.2,81.1,1.25,2017-03-31,2017-01-01,1.0
3456,TRAIN_3446,서울특별시 강남구 대치동,66,"쌍용대치아파트1동,2동,3동,5동,6동",83.5600,201707,08,139000.0,8,1983,train,2017-07-08,125.4,82.4,1.25,2017-06-30,2017-04-01,0.8
1138,TRAIN_0852,서울특별시 강남구 대치동,891-23,대우아이빌명문가(891-23),34.2000,201503,27,23250.0,8,2004,train,2015-03-27,136.6,69.8,1.75,2014-12-31,2014-10-01,0.5


In [48]:
df_gdp

,기간,GDP 성장률
0,2012-10-01,0.5
1,2013-01-01,0.9
2,2013-04-01,1.2
3,2013-07-01,0.9
4,2013-10-01,0.9
...,...,...
38,2022-04-01,0.8
39,2022-07-01,0.2
40,2022-10-01,-0.3
41,2023-01-01,0.3


In [49]:
merged_df2.sort_values(by='id')

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,train_test,transaction_date,소비심리지수,아파트 매매 실거래 가격 지수,기준금리,전분기 마지막일,기간,GDP 성장률
6002,TEST_0000,서울특별시 강남구 대치동,977,(977-),77.97,202304,10,NaN,2,2021,test,2023-04-10,97.5,98.7,3.50,2023-03-31,2023-01-01,0.3
6006,TEST_0001,서울특별시 강남구 대치동,977,(977-),59.99,202304,10,NaN,2,2021,test,2023-04-10,97.5,98.7,3.50,2023-03-31,2023-01-01,0.3
5924,TEST_0002,서울특별시 강남구 대치동,503,개포우성1,84.81,202301,13,NaN,3,1983,test,2023-01-13,76,100.9,3.50,2022-12-31,2022-10-01,-0.3
5928,TEST_0003,서울특별시 강남구 대치동,503,개포우성1,84.81,202301,19,NaN,9,1983,test,2023-01-19,76,100.9,3.50,2022-12-31,2022-10-01,-0.3
5949,TEST_0004,서울특별시 강남구 대치동,503,개포우성1,84.81,202302,15,NaN,5,1983,test,2023-02-15,82.8,99.9,3.50,2022-12-31,2022-10-01,-0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5860,TRAIN_5982,서울특별시 강남구 대치동,511,한보미도맨션2,190.47,202205,09,507500.0,8,1985,train,2022-05-09,113,106,1.50,2022-03-31,2022-01-01,0.7
5881,TRAIN_5983,서울특별시 강남구 대치동,511,한보미도맨션2,126.33,202206,24,380000.0,10,1985,train,2022-06-24,107.3,106.1,1.75,2022-03-31,2022-01-01,0.7
5882,TRAIN_5984,서울특별시 강남구 대치동,511,한보미도맨션2,126.33,202206,24,380000.0,10,1985,train,2022-06-24,107.3,106.1,1.75,2022-03-31,2022-01-01,0.7
5891,TRAIN_5985,서울특별시 강남구 대치동,511,한보미도맨션2,126.33,202207,12,380000.0,7,1985,train,2022-07-12,100.9,106.1,1.75,2022-06-30,2022-04-01,0.8


In [50]:
merged_df2.drop(['전분기 마지막일','기간'], axis = 1, inplace = True)

In [51]:
merged_df2.columns

Index(['id', 'sigungu', 'jibun', 'apt_name', 'exclusive_use_area',
       'transaction_year_month', 'transaction_day', 'transaction_real_price',
       'floor', 'year_of_completion', 'train_test', 'transaction_date',
       '소비심리지수', '아파트 매매 실거래 가격 지수', '기준금리', 'GDP 성장률'],
      dtype='object')

## 파생변수 생성
### 최근 실거래 데이터
- data leakage 방지를 위해 test 데이터의 경우 train 데이터의 최신 실거래 데이터를 사용함
- 전세 데이터와 실거래가 데이터를 살펴보던 중, 아파트 명이 다른 경우가 있어 아파트 명 전처리 후 최근 실거래가 데이터 생성함

In [52]:
jeonse = pd.read_excel('../data/아파트 전세 실거래가.xlsx')
jeonse.head()

,시군구,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),층,건축년도,도로명,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,서울특별시 강남구 대치동,개포우성1,전세,84.81,201401,6,"70,000",10,1983,선릉로 120,NaN,NaN,NaN,NaN,NaN,NaN
1,서울특별시 강남구 대치동,개포우성1,전세,84.81,201401,7,"70,000",9,1983,선릉로 120,NaN,NaN,NaN,NaN,NaN,NaN
2,서울특별시 강남구 대치동,개포우성1,전세,84.81,201401,25,"60,000",13,1983,선릉로 120,NaN,NaN,NaN,NaN,NaN,NaN
3,서울특별시 강남구 대치동,개포우성1,전세,127.61,201401,2,"96,000",10,1983,선릉로 120,NaN,NaN,NaN,NaN,NaN,NaN
4,서울특별시 강남구 대치동,개포우성1,전세,127.61,201401,7,"87,000",11,1983,선릉로 120,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
df_jeonse = jeonse[jeonse.columns[1:10]]
df_jeonse.head()

,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),층,건축년도,도로명
0,개포우성1,전세,84.81,201401,6,"70,000",10,1983,선릉로 120
1,개포우성1,전세,84.81,201401,7,"70,000",9,1983,선릉로 120
2,개포우성1,전세,84.81,201401,25,"60,000",13,1983,선릉로 120
3,개포우성1,전세,127.61,201401,2,"96,000",10,1983,선릉로 120
4,개포우성1,전세,127.61,201401,7,"87,000",11,1983,선릉로 120


In [54]:
df_jeonse['단지명'].nunique()

74

In [55]:
merged_df2['apt_name'].nunique()

70

In [56]:
lst_apt = merged_df2['apt_name'].unique()
lst_jeonse = df_jeonse['단지명'].unique()

In [57]:
diff_A_B = [item for item in lst_apt if item not in lst_jeonse]

In [58]:
# 실거래가 데이터에는 있지만 전세 데이터에는 없는 아파트
# => 해당 기간동안 재건축 진행으로 매매는 진행됐지만 전세거래는 안된 것 으로 보임
diff_A_B

['국제', '해암프리존', '(977-)']

In [59]:
diff_jeonse_apt = [item for item in lst_jeonse if item not in lst_apt]
diff_jeonse_apt

['대치동더블유타워', '하우스온더락', 'S-TOWER', '대치르엘', '대치더클래스', '아티스톤', '대치푸르지오써밋']

실거래 데이터 => 재건축 => 전세 데이터
- 국제 => 재건축 =>  SK뷰
- 해암프리존 => 재건축 => S-TOWER
- 해당 기간 재건축으로 인해 매매 거래는 있지만 전세 거래는 없음

=> 국제, 해암프리존 아파트의 경우 데이터 기간동안 재건축이 진행돼 아파트 시장 가지를 크게 변화시킬 것이라고 판단함. 이상치로 처리하여 분석에서 제외

아파트명이 다르게 나온 데이터
- 977- =>  대치르엘

In [60]:
merged_df2[merged_df2['apt_name'].str.contains('국제|해암프리존|977-')]

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,train_test,transaction_date,소비심리지수,아파트 매매 실거래 가격 지수,기준금리,GDP 성장률
248,TRAIN_0061,서울특별시 강남구 대치동,612,국제,84.69,201403,03,84500.0,3,1983,train,2014-03-03,132.6,67.3,2.50,0.9
306,TRAIN_0062,서울특별시 강남구 대치동,612,국제,84.69,201404,11,87000.0,6,1983,train,2014-04-11,117.1,67.9,2.50,0.9
308,TRAIN_0063,서울특별시 강남구 대치동,612,국제,84.69,201404,14,83600.0,2,1983,train,2014-04-14,117.1,67.9,2.50,0.9
333,TRAIN_0064,서울특별시 강남구 대치동,612,국제,84.69,201405,01,85000.0,2,1983,train,2014-05-01,116.2,67.8,2.50,0.9
336,TRAIN_0065,서울특별시 강남구 대치동,612,국제,132.53,201405,02,128750.0,8,1983,train,2014-05-02,116.2,67.8,2.50,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1210,TRAIN_0841,서울특별시 강남구 대치동,612,국제,84.69,201504,16,98000.0,3,1983,train,2015-04-16,138.7,70.6,1.75,0.8
1220,TRAIN_1725,서울특별시 강남구 대치동,983-1,해암프리존,204.38,201504,18,115000.0,8,1996,train,2015-04-18,138.7,70.6,1.75,0.8
1284,TRAIN_0842,서울특별시 강남구 대치동,612,국제,84.69,201505,11,102000.0,2,1983,train,2015-05-11,139.7,71.3,1.75,0.8
6002,TEST_0000,서울특별시 강남구 대치동,977,(977-),77.97,202304,10,NaN,2,2021,test,2023-04-10,97.5,98.7,3.50,0.3


In [61]:
# 재건축 아파트 처리
merged_df2 = merged_df2[~merged_df2['apt_name'].str.contains('국제|해암프리존')]

In [62]:
merged_df2['apt_name'] = merged_df2['apt_name'].replace('(977-)','대치르엘')

In [63]:
merged_df2[merged_df2['apt_name'].str.contains('(977-)|대치르엘')]

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,train_test,transaction_date,소비심리지수,아파트 매매 실거래 가격 지수,기준금리,GDP 성장률
6002,TEST_0000,서울특별시 강남구 대치동,977,대치르엘,77.97,202304,10,NaN,2,2021,test,2023-04-10,97.5,98.7,3.5,0.3
6006,TEST_0001,서울특별시 강남구 대치동,977,대치르엘,59.99,202304,10,NaN,2,2021,test,2023-04-10,97.5,98.7,3.5,0.3


In [64]:
df = merged_df2.copy()

In [65]:
df.shape

(6089, 16)

In [66]:
# 아파트 그룹화
df['apartment_id'] = df.groupby(['apt_name']).ngroup()

In [68]:
train_df = df[df['train_test'] == 'train'] 
test_df = df[df['train_test'] == 'test'] 

In [69]:
def get_recent_price(idx, data, current_row):
    
    # 데이터 날짜 기준으로 최근 거래된 아파트 데이터프레임 생성
    temp_df = data[
        (data['transaction_date'] < current_row['transaction_date']) &
        (data['exclusive_use_area'] == current_row['exclusive_use_area'])
    ]
    
    # 만족하는 데이터가 없다면 해당 행의 아파트 실거래가 리턴 
    if len(temp_df) == 0:
        return current_row['transaction_real_price']
        

    # 아파트 아이디 같은것 찾기
    recent_price = temp_df[(temp_df['apartment_id'] == current_row['apartment_id'])]

    # 최근 실거래가 넣기 
    if len(recent_price) > 0:
        recent_price = recent_price.iloc[-1]['transaction_real_price']
    else:
        recent_price = temp_df.iloc[-1]['transaction_real_price']

    return recent_price

# 학습 데이터에 대한 'recent_price' 계산
for idx, row in tqdm(train_df.iterrows(), total = train_df.shape[0]):
    train_df.loc[idx, 'recent_price'] = get_recent_price(idx, train_df, row)

# 테스트 데이터에 대한 'recent_price' 계산
for idx, row in tqdm(test_df.iterrows(), total = test_df.shape[0]):
    test_df.loc[idx, 'recent_price'] = get_recent_price(idx, train_df, row)  # 테스트 데이터의 'recent_price'는 학습 데이터에 기반해야 합니다.


100%|███████████████████████████████████████████████████████████████████████████████| 196/196 [00:00<00:00, 371.50it/s]


In [70]:
train_df[train_df['recent_price'].isnull()]

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,train_test,transaction_date,소비심리지수,아파트 매매 실거래 가격 지수,기준금리,GDP 성장률,apartment_id,recent_price


In [71]:
# 근소한 평수 차이로 결측값이 발생
test_df[test_df['recent_price'].isnull()]

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,train_test,transaction_date,소비심리지수,아파트 매매 실거래 가격 지수,기준금리,GDP 성장률,apartment_id,recent_price
6002,TEST_0000,서울특별시 강남구 대치동,977,대치르엘,77.9700,202304,10,NaN,2,2021,test,2023-04-10,97.5,98.7,3.5,0.3,8,NaN
6021,TEST_0016,서울특별시 강남구 대치동,891-26,대우아이빌멤버스(891-26),31.3930,202304,24,NaN,11,2003,test,2023-04-24,97.5,98.7,3.5,0.3,3,NaN
6022,TEST_0021,서울특별시 강남구 대치동,1029,대치SKVIEW,84.3382,202304,26,NaN,5,2017,test,2023-04-26,97.5,98.7,3.5,0.3,5,NaN
6046,TEST_0022,서울특별시 강남구 대치동,1029,대치SKVIEW,59.6710,202305,17,NaN,1,2017,test,2023-05-17,99.3,98.7,3.5,0.3,5,NaN
6048,TEST_0157,서울특별시 강남구 대치동,891-6,테헤란로대우아이빌(891-6),66.8300,202305,18,NaN,26,2004,test,2023-05-18,99.3,98.7,3.5,0.3,49,NaN
6065,TEST_0162,서울특별시 강남구 대치동,910-6,풍림아이원2차202동,137.4000,202305,27,NaN,6,2003,test,2023-05-27,99.3,98.7,3.5,0.3,54,NaN


<strong><font size = '4'>대우아이빌멤버스(891-26)</strong>

In [72]:
train_df[train_df['apt_name'] == '대우아이빌멤버스(891-26)']['exclusive_use_area'].unique()

array([35.796, 35.236, 42.837, 54.966, 25.513, 51.026, 36.566, 41.112,
       46.877])

In [73]:
# 31.393 -> 비슷한 평수인 35.236 최근 실거래가 사용
test_df[test_df['apt_name'] == '대우아이빌멤버스(891-26)']['exclusive_use_area'].unique()

array([35.796, 42.837, 31.393, 35.236])

In [74]:
df_jeonse[df_jeonse['단지명'] == '대우아이빌멤버스(891-26)']['전용면적(㎡)'].unique()

array([42.84, 51.03, 35.24, 35.8 , 31.39, 46.88, 25.51, 36.57, 41.11])

<strong><font size = '4'>대치SKVIEW</strong>

In [75]:
train_df[train_df['apt_name'] == '대치SKVIEW']['exclusive_use_area'].unique()

array([ 93.4   ,  84.3865, 112.0834,  93.4823, 125.3512, 112.3685])

In [76]:
# 84.3382-> 비슷한 평수인 84.3865로 최근 실거래가 사용 , 59.671 면적  -> 면적과 거래일자 비슷한 타 아파트 최근 거래데이터 사용
test_df[test_df['apt_name'] == '대치SKVIEW']['exclusive_use_area'].unique()

array([93.4   , 84.3382, 59.671 ])

In [77]:
df_jeonse[df_jeonse['단지명'] == '대치SKVIEW']['전용면적(㎡)'].unique()

array([ 84.34,  93.48,  84.39,  93.4 , 112.08, 112.37, 125.35,  59.67])

<strong><font size = '4'> 테헤란로대우아이빌(891-6) </strong>

In [78]:
train_df[train_df['apt_name'] == '테헤란로대우아이빌(891-6)']['exclusive_use_area'].unique()

array([33.11, 35.51, 59.83, 29.63, 29.02, 65.24, 31.51, 30.46, 35.61])

In [79]:
# 66.83 -> 비슷한 평수인 65.24  최근 실거래가 사용
test_df[test_df['apt_name'] == '테헤란로대우아이빌(891-6)']['exclusive_use_area'].unique()

array([29.02, 35.51, 33.11, 66.83])

In [80]:
df_jeonse[df_jeonse['단지명'] == '테헤란로대우아이빌(891-6)']['전용면적(㎡)'].unique()

array([59.83, 29.02, 33.11, 35.51, 29.63, 35.61, 65.24, 66.83, 30.46,
       31.51])

<strong><font size = '4'>풍림아이원2차202동</strong>

In [81]:
train_df[train_df['apt_name'] == '풍림아이원2차202동']['exclusive_use_area'].unique()

array([156.21, 115.22, 134.61])

In [82]:
# 137.4 -> 비슷한 평수인 134.61  최근 실거래가 사용
test_df[test_df['apt_name'] == '풍림아이원2차202동']['exclusive_use_area'].unique()

array([137.4])

In [83]:
df_jeonse[df_jeonse['단지명'] == '풍림아이원2차202동']['전용면적(㎡)'].unique()

array([156.21, 134.61, 115.22])

<strong><font size = 4> 수정된 최근 거래 데이터 함수</strong>
- 공급면적 소수점 둘째자리에서 반올림
- 전용면적 범위 늘리기

In [84]:
def get_recent_price2(idx, data, current_row):
    
    # 전용면적 범위 늘리기
    temp_df = data[
        (data['transaction_date'] < current_row['transaction_date']) &
        (data['exclusive_use_area'] - 4 < current_row['exclusive_use_area'] ) &
        (data['exclusive_use_area'] + 4 > current_row['exclusive_use_area'] )
    ]
    
    # 거래일자와 해당 거래면적에 해당되는 데이터가 없다면 현재 실거래가 넣기
    if len(temp_df) == 0:
        return current_row['transaction_real_price']

    # 아파트 아이디 같은것 찾기
    recent_price = temp_df[(temp_df['apartment_id'] == current_row['apartment_id'])]

    # 동일한 아파트 최근 실거래가 넣기
    if len(recent_price) > 0:
        recent_price = recent_price.iloc[-1]['transaction_real_price']
    else:
    # 만족하는 데이터가 없다면 거래일자와 거래면적이 비슷한 최근 아파트 실거래가 넣기
        recent_price = temp_df.iloc[-1]['transaction_real_price']

    return recent_price

# 학습 데이터에 대한 'recent_price' 계산
for idx, row in tqdm(train_df.iterrows(), total = train_df.shape[0]):
    train_df.loc[idx, 'recent_price'] = get_recent_price2(idx, train_df, row)

# 테스트 데이터에 대한 'recent_price' 계산
for idx, row in tqdm(test_df.iterrows(), total = test_df.shape[0]):
    test_df.loc[idx, 'recent_price'] = get_recent_price2(idx, train_df, row)  # 테스트 데이터의 'recent_price'는 학습 데이터에 기반해야 함


100%|███████████████████████████████████████████████████████████████████████████████| 196/196 [00:00<00:00, 253.78it/s]


In [85]:
train_df[train_df['recent_price'].isnull()]

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,train_test,transaction_date,소비심리지수,아파트 매매 실거래 가격 지수,기준금리,GDP 성장률,apartment_id,recent_price


In [86]:
test_df[test_df['recent_price'].isnull()]

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,train_test,transaction_date,소비심리지수,아파트 매매 실거래 가격 지수,기준금리,GDP 성장률,apartment_id,recent_price


### 전세 

- 최근 거래된 전세 데이터 넣기

In [87]:
lst_apt = df['apt_name'].unique()

In [88]:
apt_name = '|'.join(lst_apt)
apt_name

'동부센트레빌|하이캐슬|한보미도맨션1|테헤란로대우아이빌(891-6)|대치동우정에쉐르1|대치삼성|대치효성|은마|한보미도맨션2|선경1차(1동-7동)|개포우성1|대치아이파크|삼환SOGOOD|개포우성2|대치현대|대우아이빌멤버스(891-26)|포스코더샵|대우아이빌명문가(891-23)|쌍용대치아파트1동,2동,3동,5동,6동|풍림아이원4차(1007-2)|선경2차(8동-12동)|롯데캐슬리베|선릉역풍림아이원레몬|대치우성아파트1동,2동,3동,5동,6동,7동|쌍용대치2|풍림아이원1차101동(910-3)|롯데캐슬|아름빌(889-74)|풍림아이원아파트|선릉역대우아이빌(890-49)|래미안대치하이스턴|세영팔레스타운|동민맥스빌A동|선경3차|상지리츠빌카일룸(1009-4)|화인하이빌|동양|현대1|신성미소시티|메트로|풍림아이원2차201동|세연파크뷰|대치동우정에쉐르2(890-42)|삼성3차|한티(933-35)|대치한신휴플러스|풍림아이원3차(1007-1)|대치하나빌|하이캐슬102동|풍림아이원1차103동(910-5)|동민맥스빌B동|한양팰리스|미도맨션3차|우정에쉐르|스카이써밋아파트|대치주성|풍림아이원2차202동|대치타워|청원|래미안대치팰리스|월드빌|한티(933-0)|현대썬앤빌테헤란|대치SKVIEW|노빌리티빌리지|삼성2차|청암빌라트|대치르엘'

In [89]:
df_jeonse2 = df_jeonse[df_jeonse['단지명'].str.contains(apt_name)]

In [90]:
train_jeonse = df_jeonse2[ df_jeonse2['계약년월'] < 202301]
train_jeonse

,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),층,건축년도,도로명
0,개포우성1,전세,84.81,201401,6,"70,000",10,1983,선릉로 120
1,개포우성1,전세,84.81,201401,7,"70,000",9,1983,선릉로 120
2,개포우성1,전세,84.81,201401,25,"60,000",13,1983,선릉로 120
3,개포우성1,전세,127.61,201401,2,"96,000",10,1983,선릉로 120
4,개포우성1,전세,127.61,201401,7,"87,000",11,1983,선릉로 120
...,...,...,...,...,...,...,...,...,...
15039,현대썬앤빌테헤란,전세,36.12,202202,25,"39,900",4,2016,삼성로85길 33
15040,현대썬앤빌테헤란,전세,36.12,202202,25,"39,900",4,2016,삼성로85길 33
15041,현대썬앤빌테헤란,전세,45.89,202203,12,"58,800",7,2016,삼성로85길 33
15042,화인하이빌,전세,117.98,202206,11,"55,200",1,2004,영동대로85길 17-9


In [91]:
df_jeonse.head(4)

,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),층,건축년도,도로명
0,개포우성1,전세,84.81,201401,6,"70,000",10,1983,선릉로 120
1,개포우성1,전세,84.81,201401,7,"70,000",9,1983,선릉로 120
2,개포우성1,전세,84.81,201401,25,"60,000",13,1983,선릉로 120
3,개포우성1,전세,127.61,201401,2,"96,000",10,1983,선릉로 120


In [92]:
train_jeonse.dtypes

단지명         object
전월세구분       object
전용면적(㎡)    float64
계약년월         int64
계약일          int64
보증금(만원)     object
층            int64
건축년도         int64
도로명         object
dtype: object

In [93]:
train_jeonse['거래일자'] = pd.to_datetime(train_jeonse['계약년월'].astype(str) + train_jeonse['계약일'].astype(str), format='%Y%m%d')

In [94]:
train_jeonse['거래일자'].iloc[1]

Timestamp('2014-01-07 00:00:00')

In [95]:
train_df.dtypes

id                                object
sigungu                           object
jibun                             object
apt_name                          object
exclusive_use_area               float64
transaction_year_month            object
transaction_day                   object
transaction_real_price           float64
floor                              int64
year_of_completion                 int64
train_test                        object
transaction_date          datetime64[ns]
소비심리지수                            object
아파트 매매 실거래 가격 지수                  object
기준금리                             float64
GDP 성장률                          float64
apartment_id                       int64
recent_price                     float64
dtype: object

In [96]:
train_df['transaction_date'].iloc[1]

Timestamp('2014-01-02 00:00:00')

In [97]:
train_df.shape, train_jeonse.shape

((5893, 18), (13513, 10))

In [98]:
train_jeonse.columns

Index(['단지명', '전월세구분', '전용면적(㎡)', '계약년월', '계약일', '보증금(만원)', '층', '건축년도', '도로명',
       '거래일자'],
      dtype='object')

In [99]:
train_jeonse['보증금(만원)'] = train_jeonse['보증금(만원)'].str.strip()
train_jeonse['보증금(만원)'] = train_jeonse['보증금(만원)'].str.replace(',','').astype('int')
train_jeonse['보증금(만원)']

0        70000
1        70000
2        60000
3        96000
4        87000
         ...  
15039    39900
15040    39900
15041    58800
15042    55200
15043    77700
Name: 보증금(만원), Length: 13513, dtype: int32

In [100]:
def get_recent_jeonse(idx, current_row, jeonse_data):
   
    # 거래일자와 해당 거래면적에 해당되는 데이터 추출
    
    temp_df = jeonse_data[
        (jeonse_data['거래일자'] < current_row['transaction_date']) &
        (jeonse_data['전용면적(㎡)'] - 3 < current_row['exclusive_use_area']) &
        (jeonse_data['전용면적(㎡)'] + 3 > current_row['exclusive_use_area']) &
        (jeonse_data['단지명'] == current_row['apt_name'])
    ]
    
    # 최근 거래된 전세 데이터 넣기
    if len(temp_df) > 0:
        recent_jeonse = temp_df.iloc[-1]['보증금(만원)']
    
    # 최근 거래되지 않은 전세 데이터의 경우 None 값 처리 => 개별 데이터 확인
    else:
        recent_jeonse = None  
    
    return recent_jeonse

# 학습 데이터에 대한 'recent_jeonse' 계산
for idx, row in tqdm(train_df.iterrows(), total=train_df.shape[0]):
    train_df.loc[idx, 'recent_jeonse'] = get_recent_jeonse(idx, row, train_jeonse)

# 테스트 데이터에 대한 'recent_jeonse' 계산
for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    test_df.loc[idx, 'recent_jeonse'] = get_recent_jeonse(idx, row, train_jeonse)  # 테스트 데이터의 'recent_jeonse'는 jeonse_data에 기반해야 함.


100%|███████████████████████████████████████████████████████████████████████████████| 196/196 [00:00<00:00, 208.12it/s]


In [101]:
train_jeonse[ (train_jeonse['단지명'] == '은마') & (train_jeonse['전용면적(㎡)'] == 76.79) & (train_jeonse['계약년월'] < 201602
                                                                                   )].tail()

,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),층,건축년도,도로명,거래일자
3672,은마,전세,76.79,201512,28,45000,11,1979,삼성로 212,2015-12-28
3673,은마,전세,76.79,201512,28,42000,4,1979,삼성로 212,2015-12-28
3674,은마,전세,76.79,201512,29,46000,9,1979,삼성로 212,2015-12-29
3675,은마,전세,76.79,201512,31,43000,3,1980,삼성로 212,2015-12-31
3676,은마,전세,76.79,201512,31,40000,6,1979,삼성로 212,2015-12-31


In [102]:
train_jeonse[ (train_jeonse['단지명'] == '은마') &(train_jeonse['계약년월'] < 201602
                                                                                   )].tail()

,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),층,건축년도,도로명,거래일자
3700,은마,전세,84.43,201512,28,53000,13,1980,삼성로 212,2015-12-28
3701,은마,전세,84.43,201512,28,51000,2,1979,삼성로 212,2015-12-28
3702,은마,전세,84.43,201512,28,50000,5,1980,삼성로 212,2015-12-28
3703,은마,전세,84.43,201512,29,50000,12,1979,삼성로 212,2015-12-29
3704,은마,전세,84.43,201512,31,55000,8,1980,삼성로 212,2015-12-31


In [103]:
train_df[train_df['recent_jeonse'].isnull()].shape

(846, 19)

- 최근 거래된 전세데이터가 없는 행이 800여개 존재
- 매매가 대비 전세가의 비율의 평균을 구해 결측값 채우기

In [104]:
jeonse_to_sale_ratio = np.mean( train_df[train_df['recent_jeonse'].notnull()]['recent_jeonse'] / train_df[train_df['recent_jeonse'].notnull()]['recent_price'])
jeonse_to_sale_ratio

0.5454376464858396

<strong><font size = 4>  수정된 최근 전세 데이터 함수</strongn>
- 매매가 대비 전세가 비율 둘째자리에서 반올림

In [105]:
def get_recent_jeonse(idx, current_row, jeonse_data):
    # 전용면적 범위 늘리기
    temp_df = jeonse_data[
        (jeonse_data['거래일자'] < current_row['transaction_date']) &
        (jeonse_data['전용면적(㎡)'] - 4 < current_row['exclusive_use_area']) &
        (jeonse_data['전용면적(㎡)'] + 4 > current_row['exclusive_use_area']) &
        (jeonse_data['단지명'] == current_row['apt_name'])
    ]
    
    # 거래일자와 해당 거래면적에 해당되는 데이터가 없다면 동일 아파트 단지 최신 전세 보증금 넣기 
    # 동일한 아파트 최근 전세 보증금 넣기
    if len(temp_df) > 0:
        recent_jeonse = temp_df.iloc[-1]['보증금(만원)']
    else:
        recent_jeonse = current_row['recent_price'] * 0.55
    
    return recent_jeonse
        
    # 동일한 아파트 최근 전세 보증금 넣기
    recent_jeonse = temp_df.iloc[-1]['보증금(만원)']
    
    return recent_jeonse

# 학습 데이터에 대한 'recent_jeonse' 계산
for idx, row in tqdm(train_df.iterrows(), total=train_df.shape[0]):
    train_df.loc[idx, 'recent_jeonse'] = get_recent_jeonse(idx, row, train_jeonse)

# 테스트 데이터에 대한 'recent_jeonse' 계산
for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    test_df.loc[idx, 'recent_jeonse'] = get_recent_jeonse(idx, row, train_jeonse)  # 테스트 데이터의 'recent_jeonse'는 jeonse_data에 기반해야 함


100%|███████████████████████████████████████████████████████████████████████████████| 196/196 [00:01<00:00, 170.59it/s]


In [106]:
test_df[test_df['recent_jeonse'].isnull()]

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,train_test,transaction_date,소비심리지수,아파트 매매 실거래 가격 지수,기준금리,GDP 성장률,apartment_id,recent_price,recent_jeonse


In [107]:
train_df.sample()

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,train_test,transaction_date,소비심리지수,아파트 매매 실거래 가격 지수,기준금리,GDP 성장률,apartment_id,recent_price,recent_jeonse
2991,TRAIN_3512,서울특별시 강남구 대치동,316,은마,76.79,201703,25,115000.0,11,1979,train,2017-03-25,113.1,80.1,1.25,0.6,46,109800.0,45000.0


In [108]:
data = pd.concat([train_df,test_df], axis = 0)
data.shape

(6089, 19)

In [109]:
data = pd.concat([train_df,test_df], axis = 0)
data.shape

(6089, 19)

### 거래량
- 거래량을 통해 시장 활성화정도를 알 수 있음
- 거래량이 많을수록 가격 변동성이 커질 수 있음

In [110]:
data['transaction_date'].dtypes

dtype('<M8[ns]')

In [111]:
from datetime import datetime, timedelta

for idx, row in tqdm(data.iterrows(), total = data.shape[0]):
    # 2014년 1월 1일부터 2014년 3월 30일까지의 거래 -> train data에 한함
    if row['transaction_date'] <= datetime.strptime('2014-03-30', "%Y-%m-%d"):
        start_day = datetime.strptime('2014-01-01', "%Y-%m-%d")
        end_day = datetime.strptime('2014-03-30', "%Y-%m-%d")
        cnt = len(data[(data['transaction_date'] >= start_day) & (data['transaction_date'] < end_day) & (data['apt_name'] == row['apt_name']) ])
    
    # 2014년 3월 30일 이후 거래량
    else:
        start_day = row['transaction_date'] - timedelta(days=90)
        end_day = row['transaction_date']
        cnt = len(data[(data['transaction_date'] >= start_day) & (data['transaction_date'] < end_day) & (data['apt_name'] == row['apt_name']) ])
    
    data.loc[idx, 'transaction_cnt'] = cnt

100%|█████████████████████████████████████████████████████████████████████████████| 6089/6089 [00:18<00:00, 331.29it/s]


### 건령
- 건물의 나이가 들수록 일반적으로 가치가 감소함
- 건물의 노후화와 유지보수 비용 증가
- 대치동 특성 상, 건령이 높은 아파트의 실거래가가 높은 경우가 많은데 이러한 영향력을 보고자 파생변수 생성

In [112]:
def make_age(transaction_year, completion_year):
    age = transaction_year.year - completion_year

    return age

# 거래 시점에 따른 아파트 건령 컬럼 생성
data['age'] = data.apply(lambda row: make_age(row['transaction_date'], row['year_of_completion']), axis=1)
data['age'].value_counts()

36    405
38    386
37    322
35    281
15    267
     ... 
21     17
24     13
23     13
22      6
0       6
Name: age, Length: 45, dtype: int64

In [113]:
data.head(30)

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,train_test,transaction_date,소비심리지수,아파트 매매 실거래 가격 지수,기준금리,GDP 성장률,apartment_id,recent_price,recent_jeonse,transaction_cnt,age
0,TRAIN_0320,서울특별시 강남구 대치동,670,동부센트레빌,145.83,201401,02,180000.0,16,2005,train,2014-01-02,112.6,66.6,2.5,0.9,20,180000.0,99000.0,25.0,9
1,TRAIN_0699,서울특별시 강남구 대치동,986-14,하이캐슬,84.06,201401,02,68000.0,5,2008,train,2014-01-02,112.6,66.6,2.5,0.9,58,68000.0,37400.0,1.0,6
2,TRAIN_0705,서울특별시 강남구 대치동,511,한보미도맨션1,161.36,201401,04,155000.0,5,1983,train,2014-01-04,112.6,66.6,2.5,0.9,60,155000.0,85250.0,24.0,31
3,TRAIN_0660,서울특별시 강남구 대치동,891-6,테헤란로대우아이빌(891-6),33.11,201401,04,27000.0,20,2004,train,2014-01-04,112.6,66.6,2.5,0.9,49,27000.0,14850.0,6.0,10
4,TRAIN_0102,서울특별시 강남구 대치동,891-28,대치동우정에쉐르1,59.98,201401,04,37400.0,6,2004,train,2014-01-04,112.6,66.6,2.5,0.9,6,37400.0,20570.0,4.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,TRAIN_0001,서울특별시 강남구 대치동,503,개포우성1,127.61,201401,09,157500.0,6,1983,train,2014-01-09,112.6,66.6,2.5,0.9,0,157500.0,87000.0,18.0,31
26,TRAIN_0264,서울특별시 강남구 대치동,974,대치현대,59.82,201401,09,50000.0,22,1999,train,2014-01-09,112.6,66.6,2.5,0.9,16,81000.0,44550.0,19.0,15
27,TRAIN_0533,서울특별시 강남구 대치동,316,은마,76.79,201401,10,77300.0,2,1979,train,2014-01-10,112.6,66.6,2.5,0.9,46,74800.0,32000.0,60.0,35
28,TRAIN_0534,서울특별시 강남구 대치동,316,은마,76.79,201401,10,80000.0,6,1979,train,2014-01-10,112.6,66.6,2.5,0.9,46,74800.0,32000.0,60.0,35


### 아파트 군집화
- 유사한 특성(가격)으로 군집화하는 것이 예측 정확도를 높이는데 도움이 될 것이라고 판단함

In [114]:
from sklearn.cluster import KMeans
import numpy as np

# 아파트 별로 가격 평균값 구하기
train = data[data['train_test'] == 'train']
apt_price_df = train[['apt_name', 'transaction_real_price']]

apt_price = apt_price_df.groupby('apt_name').mean()
arr = apt_price['transaction_real_price'].to_numpy().reshape(-1, 1)

# 가격을 기준으로 아파트 군집화
k = 5  
kmeans = KMeans(n_clusters=k, n_init=10)
kmeans.fit(arr)

# 가격을 기준으로 군집의 순서를 정렬하기 위해 인덱스를 추출
sort_order = np.argsort(kmeans.cluster_centers_.flatten())

# 군집화 결과를 가격 순서대로 재할당
labels = np.zeros_like(kmeans.labels_)
for i, cluster in enumerate(sort_order):
    labels[kmeans.labels_ == cluster] = i

# 군집화 결과와 가격을 데이터에 추가
apt_price['cluster'] = labels
apt_price = apt_price.reset_index()
apt_price = apt_price[['apt_name', 'cluster']]

data = pd.merge(data, apt_price, how='left', left_on='apt_name', right_on='apt_name')

cluster_mode = data.loc[data['train_test'] == 'train', 'cluster'].mode()[0]
data['cluster'] = data['cluster'].fillna(cluster_mode)

data['cluster'].value_counts()

2.0    2494
3.0    2016
1.0    1096
0.0     474
4.0       9
Name: cluster, dtype: int64

In [115]:
data.to_csv('../data/apt_indicator.csv')

In [116]:
data.shape

(6089, 22)

In [117]:
data.columns

Index(['id', 'sigungu', 'jibun', 'apt_name', 'exclusive_use_area',
       'transaction_year_month', 'transaction_day', 'transaction_real_price',
       'floor', 'year_of_completion', 'train_test', 'transaction_date',
       '소비심리지수', '아파트 매매 실거래 가격 지수', '기준금리', 'GDP 성장률', 'apartment_id',
       'recent_price', 'recent_jeonse', 'transaction_cnt', 'age', 'cluster'],
      dtype='object')